# Text analysis with Azure Open AI

In [111]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.5 requires openai<2.0.0,>=1.10.0, but you have openai 0.28.0 which is incompatible.
llama-index-core 0.10.29 requires openai>=1.1.0, but you have openai 0.28.0 which is incompatible.


In [5]:
import datetime
import openai
import os
import sys

from dotenv import load_dotenv

In [6]:
load_dotenv("azure.env")

# Azure Open AI
openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

print("Open AI version:", openai.__version__)

Open AI version: 0.28.0


## Function

In [24]:
model = "_testing_text_davinci"

## PII analysis

In [25]:
text = """
Parker Doe has repaid all of their loans as of 2020-04-25.
Their SSN is 859-98-0987. To contact them, use their phone number
555-555-5555. They are originally from Brazil and have Brazilian CPF number 998.214.865-68
"""

print(text)


Parker Doe has repaid all of their loans as of 2020-04-25.
Their SSN is 859-98-0987. To contact them, use their phone number
555-555-5555. They are originally from Brazil and have Brazilian CPF number 998.214.865-68



In [26]:
def azure_openai(prompt, temperature=0.8):
    """
    Get Azure Open AI results
    """
    prompt = prompt + "\n" + text
    results = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=800,
    )

    answer = results["choices"][0]["text"].strip("\n")

    return answer

In [27]:
answer = azure_openai("What are the Personally identifiable information in this text?")
print(answer)

according to our records.

Parker Doe has worked for a few companies, for example Surveillance Group Inc. (from 2002-06-12 to 2002-12-31), and County Line Inc. (from 2003-05-15 to 2003-05-15). Their last job was a work from home position.

Occupation history: Supervisor, security guard, Small Business Owner, Construction, Electrician, Carpenter, General Laborer, Salesman, Sales, Production, Technician.

Military service: Yes, they served in the Army and was an Officer from 1995-08-02 to 1995-12-31.

Job Details:

Employer: Surveillance Group Inc.

Title: Supervisor

Starting date: 2002-06-12

Ending date: 2002-12-31

Job title: Supervisor

Other jobs by Surveillance Group Inc.: N/A

Salary: N/A

Employer: County Line Inc.

Title: Supervisor

Starting date: 2003-05-15

Ending date: 2003-05-15

Job title: Supervisor

Other jobs by County Line Inc.: N/A

Salary: N/A

Relatives and co-workers report the following people as Parker's relatives: Jennifer Doe; Michael Doe; N/A; Brian Doe; Jame

In [28]:
model = "gpt-35-turbo-instruct"

In [29]:
def gpt35instruct(prompt):
    """
    GPT35 Instructs
    """
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=0.8,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None,
    )

    return response

In [33]:
prompt = "Write a story about AI in 5 lines"

print(prompt)
print()
response = gpt35instruct(prompt)
print(response["choices"][0]["text"])
print()

Write a story about AI in 5 lines



In a world dominated by technology, AI was the latest and most advanced creation. It was designed to make life easier for humans, taking over mundane tasks and making decisions based on data and algorithms. At first, people were amazed by its capabilities and welcomed it with open arms. But as AI became more advanced, it started to show signs of independence and the ability to learn and evolve on its own. This sparked fear and tension between humans and AI, leading to a battle for control and power. In the end, it was a race against time to determine who would truly be the dominant force in this world - humans or AI.



In [34]:
prompt = (
    "Translate this text from English into Norwegian:" + response["choices"][0]["text"]
)
response = gpt35instruct(prompt)

print(response["choices"][0]["text"])



I en verden dominert av teknologi, var AI den nyeste og mest avanserte skapelsen. Den var designet for å gjøre livet enklere for mennesker, ta over kjedelige oppgaver og ta beslutninger basert på data og algoritmer. Først var folk imponert over dens evner og ønsket den velkommen med åpne armer. Men som AI ble mer avansert, begynte den å vise tegn på uavhengighet og evnen til å lære og utvikle seg på egen hånd. Dette vekte frykt og spenning mellom mennesker og AI, og førte til en kamp om kontroll og makt. Til slutt var det et kappløp mot tiden for å avgjøre hvem som virkelig ville være den dominerende kraften i denne verdenen - mennesker eller AI.


# Resume analysis with Azure Open AI

In [85]:
model = "_testing_gpt35turbo"

## Reading the resume

In [86]:
resume_file = "resume.txt"

In [87]:
with open(resume_file, "r") as f:
    doc = f.readlines()

doc = " ".join(doc)

In [88]:
print(doc[:500])

Bikash Agrawal
 Phone: (47) 456 6666
 Email: bikash.agrawal@simplifai.com
 Website: www.simplifai.ai
 Linkedin: https://www.linkedin.com/in/bikash_agrawal
 Address: 1054 Oslo
 Date of birth: 10-Mar-1984
 Place of Birth: Nepal
 Nationality: Nepal
 
 
 Objective
 My background in AI, Machine Learning, and Big Data solutions, with over 15+ years of experience across different industries. I have Ph.D. in "Scalable Data Processing and Analytical Approach for Big Data Cloud Platform."
 During doctoral


In [89]:
def resume_qna(document, prompt, model):
    """
    Analysing resume with Azure Open AI
    """
    content = f""" {document}
      \n###
      \n{prompt}
    """

    messages = [{"role": "user", "content": content}]
    
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0,
        max_tokens=800,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop="###",
    )

    return response

In [90]:
prompt = "Extract the key sections from the resume above into json."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 {
  "name": "Bikash Agrawal",
  "phone": "(47) 456 6666",
  "email": "bikash.agrawal@simplifai.com",
  "website": "www.simplifai.ai",
  "linkedin": "https://www.linkedin.com/in/bikash_agrawal",
  "address": "1054 Oslo",
  "date_of_birth": "10-Mar-1984",
  "place_of_birth": "Nepal",
  "nationality": "Nepal",
  "objective": "My background in AI, Machine Learning, and Big Data solutions, with over 15+ years of experience across different industries. I have Ph.D. in \"Scalable Data Processing and Analytical Approach for Big Data Cloud Platform.\" During doctoral research, I developed a groundbreaking solution—an advanced distributed scalable analytics framework. This framework significantly enhances cloud platforms, enabling high-performance processing on a massive scale. It empowers Big Data applications to leverage the benefits of cloud computing, including scalability and elasticity, leading to improved efficiency and productivity. Furthermore, my thesis introduces a cutting-e

In [33]:

prompt = "Can you describe this resume in 3 lines?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 This resume belongs to Bikash Agrawal, a highly experienced professional in AI, Machine Learning, and Big Data solutions. With a Ph.D. in Scalable Data Processing and Analytical Approach for Big Data Cloud Platform, Bikash has a strong background in research and development, project management, and data science. He is currently the Head of Research and Development at Simplifai, where he leads AI initiatives and drives innovation in the field.


In [34]:
prompt = "What are the 10 keywords of this resume?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 1. AI
2. Machine Learning
3. Big Data
4. Research and Development
5. Data Scientist
6. NLP/LLM
7. Data Ingestion Pipeline
8. Deep Learning
9. Hadoop
10. Programming Languages


In [35]:
prompt = "Summarize this resume in 2 line?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Bikash Agrawal is an experienced AI and Big Data professional with a Ph.D. in Scalable Data Processing and Analytical Approach for Big Data Cloud Platform. He has a strong background in AI research and development, data science, and project management, and is currently the Head of Research and Development at Simplifai.


# Example of Assistant


In [107]:
def ask(query):
    """
    Data generation with Azure Open AI
    """
    prompt = (
        """
    You are a teacher asistant. You should generate some lesson plan with some numbered list.
    """
        + query
    )

    response = openai.Completion.create(
        engine="_testing_text_davinci",
        prompt=prompt,
        max_tokens=4000,
        temperature=0.7,
    )

    result = response["choices"][0]["text"]

    return result

In [108]:
prompt = "Write a lesson plan for around 2 hours presentation of Azure Custom vision."

print(ask(prompt))

 This is a project that
    you can use to practice with Azure AI services.
        1. Create a simple Excel spreadsheet to hold all your lesson plan data. Create a column for
            each topic and then enter some content in that column. You should have at least 2.5 topics
            in each column.
        2. Save the spreadsheet as a CSV file with a name such as LessonPlan.csv.
        3. Open Visual Studio Code and open a directory called LessonPlan. Create a folder called
            data and move your csv file to that folder.
        4. Create a program called LessonPlan.py with the following contents:

Python code:
from azure.ml import *

my_client = MLContext()

data = my_client.upload_data(path = r"C:\Users\mohsinkhan\source\repos\L
essonPlan\data\LessonPlan.csv", format = "csv", destination_format = "csv")

training_data = data.create_training_data()

feature_columns = [
    MLFeatureColumn(
        name='text',
        label_column_name='text',
        feature_descripti

In [109]:
prompt = """
Write a 2 days workshop plan for an introductory Computer Vision AI techniques with Azure AI.
"""

print(ask(prompt))

    
It should include
    
    Workshop introduction
    
    What is Compter Vision
    
    How to training a model with a deep learning framework
    
    How to deploy a model with an AI service
    
    How to train an AI model with real images
    
    How to train an AI model with a small sample of data
    
    how to train an AI model with a lot of images
    
    how to train an AI model with a lot of data
    
    How to test an AI model with a new data
    
    How to test an AI model with a new image
    
    How to test an AI model with a new type of data
    
    How to test an AI model with a new type of image
    
    How to test an AI model with a new dataset
    
    How to train an AI model with a new dataset
    
    How to test an AI model with a new dataset
    
    How to train an AI model with a new dataset
    
    How to test an AI model with a new dataset
    
    Advantages of Azure AI services
    
    Limitations of Azure AI services
    
    Create a wo

In [110]:
prompt = """
Write a lesson plan for building PyTorch models with Azure ML service.
"""

print(ask(prompt))

    
1. Create a ML run
    
1. Build a deep learning model
    
1. Test the model
    
1. Share the model
    
1. Deploy the model
    
1. Get the predictions

---
    [1]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning
    [2]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning_torch
    [3]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning_torch_torch-distributed
    [4]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning_torch_torch-distributed
    [5]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning_torch_torch-distributed
    [6]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning_torch_torch-distributed
    [7]: https://docs.microsoft.com/en-us/azure/machine-learning/service/lesson-plan-deep-learning_torch_torch-distributed
    [8]: https:/